# Gaussian Mixture Models

#### Author: Atharv Subhekar
#### CWID: 20015840

**Notation**:

- n    : number of training points
- f    : number of features
- k    : number of gaussians
- X    : n x f matrix of training data
- r    : n x k matrix of membership weights
- pie  : k x 1 matrix of mixture weights (gaussian prior probabilities)
- mu   : k x f matrix containing the means of each gaussian
- sigma: f x f x k tensor of covariance matrices

## Importing Libraries

In [35]:
import numpy as np
from scipy.stats import multivariate_normal

## Data Preprocessing

In [36]:
# training data
X = np.genfromtxt('points.dat.txt', delimiter = ' ')
print("n x f matrix of training data :",X.shape)

n x f matrix of training data : (1000, 2)


## Question 1

Expectation: Complete the function [𝑟] = 𝐸𝑥𝑝𝑒𝑐𝑡𝑎𝑡𝑖𝑜𝑛(𝑋, 𝑘, 𝜋, 𝜇, Σ). This function takes in a set
of parameters of a gaussian mixture model, and outputs the membership weights of each data point

In [37]:
def Expectation(X, k, pie, mu, sigma):
    # initializing the weights
    r = np.zeros((X.shape[0], k))
    # Calculating weights
    for i in range(k):
        probability= multivariate_normal.pdf(X, mean = mu[i, :])
        r[:, i] = pie[i] * probability
        r = r / np.sum(r, axis = 1, keepdims = True)
        
    return r

## Question 2

Maximization of Means: Complete the function [𝜇] = 𝑀𝑎𝑥𝑖𝑚𝑖𝑧𝑒𝑀𝑒𝑎𝑛(𝑋, 𝑘, 𝑟). This function
takes in the training data along with the membership weights, and calculates the new maximum
likelihood mean for each gaussian

In [43]:
def MaximizeMean(X, k, r):
    # Storing the mean and finding the maximum value of mean
    mu = []
    for i in range(k):
        product = np.dot(r[:, i], X)
        product = product / np.sum(r[:, i])
        mu.append(product)

    mu = np.array(mu)
    return mu

## Question 3

Maximization of Covariances: Complete the function [Σ] = 𝑀𝑎𝑥𝑖𝑚𝑖𝑧𝑒𝐶𝑜𝑣𝑎𝑟𝑖𝑎𝑛𝑐𝑒(𝑋, 𝑘, 𝑟, 𝜇).
This function takes in the training data along with membership weights and means for each gaussian,
and calculates the new maximum likelihood covariance for each gaussian

In [44]:
def MaximizeCovariance(X, k, r, mu):
    # initializing covariances
    sigma = np.random.rand(2, 2, 3)
    # maximizing covariances
    for i in range(k):
        prod = np.dot(X - mu[i], (X - mu[i]).T)
        mul = np.sum(r[:, i] * prod)
        sigma[:, :, i] = mul / np.sum(r[:, i])
    return sigma

## Question 4

Maximization of Mixture Weights: Complete the function [𝜋] = 𝑀𝑎𝑥𝑖𝑚𝑖𝑧𝑒𝑀𝑖𝑥𝑡𝑢𝑟𝑒𝑠(𝑘, 𝑟).
This function takes in the membership weights, and calculates the new maximum likelihood
mixture weight for each gaussian

In [45]:
def MaximizeMixtures(k, r):
    # initializing mixtures
    pi = []
    # maximizing mixtures
    for i in range(k):
        pi.append(np.sum(r[:, i]) / r.shape[0])
    # converting list to numpy array
    pi = np.array(pi)
    return pi

## Question 5

EM: Put everything together and implement the function [𝜋, 𝜇, Σ] = 𝐸𝑀(𝑋, 𝑘, 𝜋% , 𝜇% , Σ% , 𝑛𝐼𝑡𝑒𝑟).
This function runs the EM algorithm for 𝑛𝐼𝑡𝑒𝑟 steps/iterations and returns the parameters of the
underlying GMM.

In [48]:
def EM(X, k, pi, mu, sigma, Iters):
    # iterating for nIter times
    while Iters > 0:
        #obtaining the membership weights
        r = Expectation(X, k, pi, mu, sigma)
        # obtaining maximized means
        mu = MaximizeMean(X, k, r)
        # obtaining maximized covariances
        sigma = MaximizeCovariance(X, k, r, mu)
        # obtaining maximized mixtures
        pi = MaximizeMixtures(k, r)
        Iters -= 1
    return pi, mu, sigma

## Main Block

In [49]:
# initializing parameters
mu0 = np.random.rand(3, 2)
pie0 = np.random.rand(3, 1)
sigma0 = np.random.rand(2, 2, 3)
k = 3
# calling EM function
pi, mu, sigma = EM(X, k, pie0, mu0, sigma0, 1)
# printing results
print("Shape of Mixture Weights:", pi.shape)
print("\n", pi)

print("\n Gaussian Mean")
print("Shape: ", mu.shape)
print(mu)

print("\n Tensor of Covariances")
print("Shape: ", sigma.shape)
print(sigma)

Shape of Mixture Weights: (3,)

 [0.98584217 0.00400316 0.01015467]

 Gaussian Mean
Shape:  (3, 2)
[[-0.83516457 -0.60126417]
 [ 0.40139547  0.72815004]
 [ 0.12850757  0.75262899]]

 Tensor of Covariances
Shape:  (2, 2, 3)
[[[-4.80449915e-14 -5.24125039e-13  4.14890253e-13]
  [-4.80449915e-14 -5.24125039e-13  4.14890253e-13]]

 [[-4.80449915e-14 -5.24125039e-13  4.14890253e-13]
  [-4.80449915e-14 -5.24125039e-13  4.14890253e-13]]]
